In [29]:
# Import necessary libraries

import json
import joblib
from collections import defaultdict
from sentence_transformers import SentenceTransformer
import numpy as np
# import faiss
import scipy.sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [30]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [31]:
import requests
url = 'https://northeastern-my.sharepoint.com/personal/mhatre_ag_northeastern_edu/_layouts/15/download.aspx?UniqueId=3e78d9923545475397681c5e8a47383d&e=gTeFhD'  # Replace with actual link
r = requests.get(url, allow_redirects=True)
url1 = 'https://northeastern-my.sharepoint.com/personal/mhatre_ag_northeastern_edu/_layouts/15/download.aspx?UniqueId=a6e25a75d96e42139fc5b23ba7a7bc1b&e=S9FI43'  # Replace with actual link
r = requests.get(url1, allow_redirects=True)

## Step 1: Join business dataset with reviews dataset. Tokenize reviews and convert to lowercase

In [32]:
# Store reviews for each business
import random
business_information = defaultdict(lambda: {"reviews": []})

with open("/content/drive/MyDrive/yelp_academic_dataset_review.json", "r", encoding="utf-8") as file:
    for line in file:
        # Keep only 10% of the lines
        if random.random() <= 0.10:
            curr = json.loads(line)
            business_information[curr["business_id"]]["reviews"].append(curr["text"])

In [39]:
# Extract business-specific data and store in dictionary
with open("/content/drive/MyDrive/yelp_academic_dataset_business.json", "r", encoding="utf-8") as file:

    for line in file:

        curr = json.loads(line)
        business_information[curr["business_id"]]["name"] = curr["name"]
        business_information[curr["business_id"]]["address"] = curr["address"]
        business_information[curr["business_id"]]["city"] = curr["city"]
        business_information[curr["business_id"]]["state"] = curr["state"]
        business_information[curr["business_id"]]["latitude"] = curr["latitude"]
        business_information[curr["business_id"]]["longitude"] = curr["longitude"]
        business_information[curr["business_id"]]["stars"] = curr["stars"]
        business_information[curr["business_id"]]["categories"] = curr["categories"]

In [34]:
# Tokenize reviews and convert all tokens to lowercase
tokenized_business_information = defaultdict(lambda: {"reviews": []})

for id_ in business_information.keys():

    for review in business_information[id_]["reviews"]:

      if random.random() <= 0.10:

        tokenized_business_information[id_]["reviews"].append([word.lower() for word in word_tokenize(review)])

In [35]:
# Write joined, tokenized business/review data to a file
with open('business_with_reviews.json', 'w') as f:
    for key, value in tokenized_business_information.items():
        json.dump({key: value}, f)
        f.write('\n')

## B+

2 Problems:
1. Can't extract user lat or lon
2. Have to hardcode every bordering state

In [64]:
user_state = "CA"
user_lat = 37.77   # San Francisco
user_lon = -122.42
# Hardcode every brodering state
bordering_states = {
    'AL': ['FL', 'GA', 'MS', 'TN'],
    'AK': [],
    'AZ': ['CA', 'CO', 'NM', 'NV', 'UT'],
    'AR': ['LA', 'MO', 'MS', 'OK', 'TN', 'TX'],
    'CA': ['AZ', 'NV', 'OR'],
    'CO': ['AZ', 'KS', 'NE', 'NM', 'OK', 'UT', 'WY'],
    'CT': ['MA', 'NY', 'RI'],
    'DE': ['MD', 'NJ', 'PA'],
    'FL': ['AL', 'GA'],
    'GA': ['AL', 'FL', 'NC', 'SC', 'TN'],
    'HI': [],
    'ID': ['MT', 'NV', 'OR', 'UT', 'WA', 'WY'],
    'IL': ['IA', 'IN', 'KY', 'MO', 'WI'],
    'IN': ['IL', 'KY', 'MI', 'OH'],
    'IA': ['IL', 'MN', 'MO', 'NE', 'SD', 'WI'],
    'KS': ['CO', 'MO', 'NE', 'OK'],
    'KY': ['IL', 'IN', 'MO', 'OH', 'TN', 'VA', 'WV'],
    'LA': ['AR', 'MS', 'TX'],
    'ME': ['NH'],
    'MD': ['DE', 'PA', 'VA', 'WV'],
    'MA': ['CT', 'NH', 'NY', 'RI', 'VT'],
    'MI': ['IN', 'OH', 'WI'],
    'MN': ['IA', 'ND', 'SD', 'WI'],
    'MS': ['AL', 'AR', 'LA', 'TN'],
    'MO': ['AR', 'IA', 'IL', 'KS', 'KY', 'NE', 'OK', 'TN'],
    'MT': ['ID', 'ND', 'SD', 'WY'],
    'NE': ['CO', 'IA', 'KS', 'MO', 'SD', 'WY'],
    'NV': ['AZ', 'CA', 'ID', 'OR', 'UT'],
    'NH': ['MA', 'ME', 'VT'],
    'NJ': ['DE', 'NY', 'PA'],
    'NM': ['AZ', 'CO', 'OK', 'TX', 'UT'],
    'NY': ['CT', 'MA', 'NJ', 'PA', 'VT'],
    'NC': ['GA', 'SC', 'TN', 'VA'],
    'ND': ['MN', 'MT', 'SD'],
    'OH': ['IN', 'KY', 'MI', 'PA', 'WV'],
    'OK': ['AR', 'CO', 'KS', 'MO', 'NM', 'TX'],
    'OR': ['CA', 'ID', 'NV', 'WA'],
    'PA': ['DE', 'MD', 'NJ', 'NY', 'OH', 'WV'],
    'RI': ['CT', 'MA'],
    'SC': ['GA', 'NC'],
    'SD': ['IA', 'MN', 'MT', 'ND', 'NE', 'WY'],
    'TN': ['AL', 'AR', 'GA', 'KY', 'MO', 'MS', 'NC', 'VA'],
    'TX': ['AR', 'LA', 'NM', 'OK'],
    'UT': ['AZ', 'CO', 'ID', 'NV', 'NM', 'WY'],
    'VT': ['MA', 'NH', 'NY'],
    'VA': ['KY', 'MD', 'NC', 'TN', 'WV'],
    'WA': ['ID', 'OR'],
    'WV': ['KY', 'MD', 'OH', 'PA', 'VA'],
    'WI': ['IA', 'IL', 'MI', 'MN'],
    'WY': ['CO', 'ID', 'MT', 'NE', 'SD', 'UT']
}

In [65]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    angle = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
    dist = 2 * asin(sqrt(angle))
    return dist * 6371

In [66]:
def extract_2nd(a):
    return a[1]

def rank_by_location(query, state_u, lat_u, lon_u):
    restaurant_values = []
    allowed_states = [state_u] + bordering_states.get(state_u, [])

    for bid, info in business_information.items():
        # Check if any info is empty
        lat, lon = info.get("latitude"), info.get("longitude")
        if lon is None or lat is None:
            continue
        if "reviews" not in info or not info["reviews"]:
            continue
        # Check if the state is close, if not skip
        if info.get("state") not in allowed_states:
            continue

        # Find distance
        distance = haversine(lat_u, lon_u, lat, lon)
        # Points system, subject to change, penalty for far restaurants
        value = info["stars"] - 0.005 * distance
        restaurant_values.append((bid, value))

    return sorted(restaurant_values, key=extract_2nd, reverse=True)[:10]

In [68]:
results = rank_by_location("vegan", user_state, user_lat, user_lon)

for n, value in results:
    name = business_information[n]["name"]
    print(f"{name} ({n}): score = {value:.2f}")

SPE Painting (dytaCbK9750lx2PehZ7UHQ): score = 3.65
Tributary Whitewater Tours (VMP_tCZXcuJuiNUCBshd8w): score = 3.65
Eastside Deli (D5n8XvQqC_vzOCi63wNChQ): score = 3.65
Raft California (_dLINFtHKdhescCZiPkPlA): score = 3.64
Saturno Home Inspection (F4cezLy3xwbzCXG4lww74w): score = 3.64
Tahoe Meadows (EUvK_xnJb_ZbMZ1T5uTBUQ): score = 3.62
Sterling Detail (s9qQ2WAvlsckxabl4AEO4w): score = 3.59
HydroClean Carpets (oLWDXU_zB3_X3Yo2OJ0UXQ): score = 3.59
Greg's Small Engine (Z3EgjzkQ7azKPxlsMpRnGw): score = 3.59
Wedge on Wheels (zGt9mEjNHiuJxcuDwqZtRw): score = 3.59


## Step 2: Output scores for TFIDF ranker

In [ ]:
# Read in business reviews from JSON file
business_reviews = dict()

with open('business_with_reviews.json', 'r') as f:

    for line in f:

        record = json.loads(line)
        business_reviews.update(record)

In [ ]:
# Test one query on the entire dataset, check results
business_ids = list(business_reviews.keys())
corpus = []
for id_ in business_ids:

    # Each business review is condensed down into a single, long sentence
    # so that TFIDF and other embedding techniques could be applied
    corpus.append(" ".join(" ".join(tokens) for tokens in business_reviews[id_]["reviews"]))

In [ ]:
# Create TFIDF vector and train vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# Persist vector and model
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')
scipy.sparse.save_npz("tfidf_matrix.npz", tfidf_matrix)

In [ ]:
# Reinitialize vectorizer and tfidf_matrix if session is lost
vectorizer = joblib.load('tfidf_vectorizer.joblib')
tfidf_matrix = scipy.sparse.load_npz("tfidf_matrix.npz")

In [ ]:
# Transform query so that it fits into TFIDF vector space
query = "cafe with great lattes in san francisco"
query_vec = vectorizer.transform([query])

# Use cosine similarity to find most similar document for the query
similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

In [ ]:
# Match up business ids with similarity scores
scores = []

n = len(business_ids)
for i in range(n): scores.append((business_ids[i], similarities[i]))

# Sort based on score; sort in descending order
scores.sort(key = lambda x: x[1], reverse=True)

In [ ]:
# Print out top 10 results
for id, _ in scores[:10]:

    print(business_information[id]["name"])

## Step 3: Create/Store Embeddings and Use FAISS to generate scores, Compare to token-matching ranker

In [ ]:
# Use transformer architecture to create dense vector embeddings
# from query and restaurant reviews
# all = general purpose transformer
# MiniLM = lightweight transformer similar to BERT
# L6 = six layers
# v2 = version two
model = SentenceTransformer("all-MiniLM-L6-v2")

# Create embeddings from the reviews
embeddings = model.encode(corpus, convert_to_numpy=True)
dimension = embeddings.shape[1]

# Create FAISS index for similarity search
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [ ]:
# Persist index to file
faiss.write_index(index, "business_index.faiss")

In [ ]:
# Read index back from file if session is lost
index = faiss.read_index("business_index.faiss")

In [ ]:
# Check results of test query
query = "cafe with great lattes in san francisco"
query_vec = model.encode([query], convert_to_numpy=True)

# Get top k results
k = 50
D, I = index.search(query_vec, k)
top_businesses = [business_ids[i] for i in I[0]]

for id in top_businesses:

    print(business_information[id]["name"])